학습용 영어-프랑스어 병렬 문장 데이터 준비
개념: 
   - 입력(영어)과 출력(프랑스어) 쌍으로 구성
   - 디코더 입력에는 시작 토큰(\t), 타겟에는 종료 토큰(\n) 추가
 설명:
   - input_texts: 인코더에 입력될 영어 문장
   - target_texts: 디코더가 생성해야 할 프랑스어 문장 (전처리 포함)

In [1]:
import numpy as np
import tensorflow as tf

data_pairs = [
    ("Hello", "Bonjour"),
    ("How are you", "Comment allez-vous"),
    ("Good morning", "Bonjour matin"),
    ("Thank you", "Merci"),
]

# 입력과 타켓을 분리
input_texts = []    
target_texts = []

# 입력에는 시작토큰, 타겟에는 종료토큰을 부여
for eng, fra in data_pairs:
    input_texts.append(eng)
    # 디코더 입력 '\t'(시작), 디코더 출력 '\n'(종료)
    target_texts.append(f'\t{fra}\n')
for i in range(len(input_texts)):
    print(f"입력:{input_texts[i]:20s} --> 타겟 : {target_texts[i]}")

입력:Hello                --> 타겟 : 	Bonjour

입력:How are you          --> 타겟 : 	Comment allez-vous

입력:Good morning         --> 타겟 : 	Bonjour matin

입력:Thank you            --> 타겟 : 	Merci



문자 단위 사전(vocabulary) 생성 및 정수 인덱스 변환
- 개념:
    - 각 문자를 고유한 정수로 매핑
    - 입력과 타겟의 사전은 별도 관리
    - 원-핫 인코딩으로 신경망 입력 형태 생성
- 설명:
    - input_characters: 영어 문장에 등장하는 모든 고유 문자
    - target_characters: 프랑스어 문장 + 특수 토큰(\t, \n)
    - encoder_input_data: 3D 배열 (샘플, 시퀀스 길이, 문자 사전 크기)

In [ ]:
# 문자 인덱싱 및 원-핫 인코딩
# 입력과 타켓의 고유한 문자 수집
input_characters = set()
target_characters = set()

for text in input_texts:
    for char in text:
        input_characters.add(char)

# set([chat for char in text for text in input_texts])
